### Transform csv files into LSTM data with specifiable window and shift

Need to provide:
- Path of the training data
- Path of the validation data
- Path of a pipeline
- filenames to save the generated training and validation data to
- The different shifts and windows used. These are the same for the training and validation data

In [ ]:
import numpy as np
from joblib import load
import json

In [ ]:
from readIn import readIn

In [ ]:
trainDataPath = "X:\\KI Praktikum\\validate_Data\\training_data"
valDataPath = "X:\\KI Praktikum\\validate_Data\\validation_data"
pathname = "X:\\KI Praktikum\\pipeline.p"
filenames_train = [["X:\\KI Praktikum\\validate_Data\\training_data\\train_s5_w50_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s5_w50_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s6_w50_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s6_w50_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s7_w50_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s7_w50_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s5_w60_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s5_w60_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s6_w60_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s6_w60_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s7_w60_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s7_w60_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s5_w70_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s5_w70_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s6_w70_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s6_w70_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\training_data\\train_s7_w70_x.json", "X:\\KI Praktikum\\validate_Data\\training_data\\train_s7_w70_y.json"]]
filenames_val = [["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s5_w50_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s5_w50_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s6_w50_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s6_w50_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s7_w50_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s7_w50_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s5_w60_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s5_w60_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s6_w60_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s6_w60_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s7_w60_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s7_w60_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s5_w70_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s5_w70_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s6_w70_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s6_w70_y.json"],
                   ["X:\\KI Praktikum\\validate_Data\\validation_data\\val_s7_w70_x.json", "X:\\KI Praktikum\\validate_Data\\validation_data\\val_s7_w70_y.json"]]
shift = [5, 6, 7]
window = [50, 60, 70]

In [ ]:
# generateDataSource() -> 3D array von den Daten
### window  represents time period by each entry in the buffer
### Shift represents the jump from value to next one in the buffer
### sampling rate

def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1, sample_rate=1):
    #subsequence_len= (window -1) *shift + 1
    subsequence_len= (window) *shift
    Signal_Length = signal_input.shape[0]
    num_samples = 1 + int((Signal_Length - subsequence_len) / sample_rate)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = np.asarray([signal_input[i*sample_rate + j * shift] for j in range(0,window)])
        y[i] = signal_output[i*sample_rate + (window-1) * shift :i*sample_rate+ (window-1) * shift + output_length]
    return x, y

In [ ]:
# Using Mustafas function
def SaveJsonWindows(filenames, shift, window, x_transform, y_train, x_validation = None, y_validation = None):
    i = 0
    if x_validation != None:
        x_in = np.concatenate((x_transform, x_validation))
    else: 
        x_in = x_transform
    if y_validation != None:
        y_in = np.concatenate((y_train, y_validation))
    else: 
        y_in = y_train

    for w in window:
        for s in shift:
            x_cv, y_cv = generateDataSource(signal_input = x_in, signal_output=y_in, window = w, shift=s, sample_rate=1)
            out_x = open(filenames[i][0], mode="x")
            json.dump(x_cv.tolist(), out_x)
            out_x.close()
            out_y = open(filenames[i][1], mode="x")
            json.dump(y_cv.tolist(), out_y)
            out_y.close()
            i += 1

In [ ]:
def OpenJsontoArr(path):
    file = open(path)
    x_3d = json.load(file)
    file.close()
    x_3d = np.asarray(x_3d)
    return x_3d

In [ ]:
df_train = readIn(trainDataPath, ipo=False, safeSingleCsv = False)
df_val = readIn(valDataPath, ipo=False, safeSingleCsv = False)

In [ ]:
print(df_train.head(5))
print(df_val.head(5))

In [ ]:
df_train["welle_z_ipo"] = 1000*df_train["welle_z_iterpolated"]
df_val["welle_z_ipo"] = 1000*df_val["welle_z_iterpolated"]
df_val = df_val.drop(columns=["welle_z_iterpolated"])
df_train = df_train.drop(columns=["welle_z_iterpolated"])

In [ ]:
y_train = df_train["welle_z_ipo"]
parameters = ["t_bett", "t_motor", "t_spindle", "M8", "M121", "M127", "M7"]
x_train = df_train[parameters].to_numpy()

y_val = df_val["welle_z_ipo"]
x_val = df_val[parameters].to_numpy()

In [ ]:
newpipeline = load(pathname)
x_transform = newpipeline.fit_transform(x_train)
x_test = newpipeline.transform(x_val)
newpipeline.steps[2][1].explained_variance_ratio_

In [ ]:
SaveJsonWindows(filenames=filenames_train, shift=shift, window=window, x_transform=x_transform, y_train=y_train)
SaveJsonWindows(filenames=filenames_val, shift=shift, window=window, x_transform=x_test, y_train=y_val)
